In [1]:
#comments beginning with #BEE were written by bee martin

In [2]:
import math
import astropy
from astropy.io import ascii
import numpy as np
import emcee
from scipy.optimize import minimize
from numpy.random import normal
from numpy.random import uniform
import matplotlib as mpl
import matplotlib.pyplot as plt
import palettable
import richardsplot as rplot
%matplotlib inline
import random
from matplotlib import rc
import pandas as pd
rc('text', usetex=False)

## open file with photo-z PDF redshift bins

In [3]:
#BEE: read in table of redshifts and save the 'zshifts' column as a variable named zshifts
#BEE: zshifts is a list of redshifts from 0.4 to 4.0
#GTR: This is just a list of redshift bins

In [4]:
zshifts_Table = ascii.read('fittingS82_zshifts.dat', format='csv')
zshifts = zshifts_Table['zshifts']

## open file with regression values

In [5]:
#BEE: create an array of sdss features
#BEE: read in table of regression values, create array of zeros with shape(features, redshifts)
#BEE: fill array of zeros with data from regression values table
#GTR: These are the mean colors and DCR slopes for the above redshift bins

In [6]:
sdss_features = ['u-g', 'g-r', 'r-i', 'i-z']
sdss_features_dcr = ['u-g', 'g-r', 'r-i', 'i-z', 'u-slope', 'g-slope']

color_fit_Table = ascii.read('fittingS82_zshiftfit.dat')
color_fit_Table.remove_column('col1')
color_fit = np.zeros((len(sdss_features), len(zshifts)))
color_fit_dcr = np.zeros((len(sdss_features_dcr), len(zshifts)))
for i in range(len(sdss_features)):
    for j in range(len(zshifts)):
        color_fit[i,j] = np.asarray(color_fit_Table[i][j])

for i in range(len(sdss_features_dcr)):
    for j in range(len(zshifts)):
        color_fit_dcr[i,j] = np.asarray(color_fit_Table[i][j])

## open file with regression covariance values

In [7]:
#BEE: read in regression covariance data
#BEE: create array of zeros with shape (features, features, redshifts), fill it with covariance table data
#GTR: These are the covariances between each of the above parameters at each redshift

In [8]:
color_covariance_Table = ascii.read('fittingS82_zshiftcovariance.dat')
color_covariance_Table.remove_column('col1')
color_covariance_Table.remove_column('col2')
color_covariance = np.zeros((len(sdss_features), len(sdss_features), len(zshifts)))
color_covariance_dcr = np.zeros((len(sdss_features_dcr), len(sdss_features_dcr), len(zshifts)))  
l = 0
for i in range(len(sdss_features_dcr)):
    for j in range(len(sdss_features_dcr)):
        for k in range(len(zshifts)):
            color_covariance_dcr[i,j,k] = np.asarray(color_covariance_Table[l][k])
        l += 1
color_covariance = color_covariance_dcr[:4, :4, :]
#print(color_covariance_dcr)
#print(color_covariance)

## open file with the simulated quasar true values

In [9]:
#BEE: Read in simulated "true" quasar data
#GTR: These are simulated quasars with simulated parameters (and their errors)

In [ ]:
test_quasars0 = ascii.read('random_quasars.dat')
test_quasars = ascii.read('random_quasars100k.dat')[:1000]
print(test_quasars.keys())

['zspec', 'u-g', 'g-r', 'r-i', 'i-z', 'u-slope', 'g-slope', 'uerr', 'gerr', 'rerr', 'ierr', 'zerr', 'u-slopeerr', 'g-slopeerr']


## define the observations

In [ ]:
#BEE: simulate airmass observations in u ang g
#GTR: We ignore the next cell?

In [ ]:

astrometric_error = [0.035,0.025]  #[u-band error, g-band error]

airmasses = uniform(low=1.0, high=1.3, size=50)
airmasses = np.append(airmasses, uniform(low=1.3, high=2.0, size=14))

filters = np.tile(['u', 'g'], int(len(airmasses)/2))


In [ ]:
#BEE: this cell will take observations from the OpSim rather than simulating them
#GTR: Not sure exactly where this opSim information comes from.  Weixiang?
#id.csv is just an indexed list of RA and Dec
#dcr_all.csv is a list of observation parameters for each of those IDs
#this includes airmass and filter, which is all that we use right now?
#It seems that right now a random object is being chosen?

In [ ]:
astrometric_error = [0.035, 0.025]
#astrometric_error = np.multiply(astrometric_error, [2,2])
print(astrometric_error)
# Weixiang: import opsim cadence after fix for python2
ids = pd.read_csv('id.csv')
cad = pd.read_csv('dcr_all.csv')

#pick random object's cadence
random_cadence = random.randint(0,max(cad['id']))
# assign the cadence of random object to dcr_0
dcr_0 = cad[cad['id'] == random_cadence].copy()
obs_g = dcr_0[dcr_0['filter'] == 'g']
obs_u = dcr_0[dcr_0['filter'] == 'u']
obs = np.concatenate((obs_g, obs_u))

### Orginal code to import cadence
# dcr = np.load('dcr.npz')
# print(list(dcr.keys()))
# dcrra_dec = dcr['ra_dec']
# dcrdata = dcr['data']
# print(dcrra_dec[0])
# obs_g = dcrdata[0][dcrdata[0]['filter']=='g']
# obs_u = dcrdata[0][dcrdata[0]['filter']=='u']
# obs = np.concatenate((obs_g, obs_u))

[0.035, 0.025]


GTR: (24 July 2020) I don't recall what these comments are about.  Should take another look at them.

GTR: Split out cell that defines airmasses.  Just define one at a time.  Predefine the experiments and comment out the ones being run each time.  Make sure that the output files are unique for each experiment.

GTR: Run colors only and colors+normal DCR just once.  We don't need to run those again.  But those can be the first 2 "experiments".

In [ ]:
#GTR: Extract the airmass and filters for each observation

In [ ]:
# Weixiang: modified the item index to match the order of columns in new file
airmasses = np.array([item[3] for item in obs])
filters = np.array([item[5] for item in obs])

#airmasses_long = np.append(airmasses, [1.6, 1.6])
#filters_long = np.append(filters, ['g', 'g'])
#airmasses_twilight = np.append(airmasses, [2.0, 2.0])
#filters_twilight = np.append(filters, ['g', 'g'])

BEE: The next cell is a switch that lets you choose the experiment to run.  There are 2 types of experiments: 'substitution' and 'addition'.  Change the string in the cell to either 'substitution' or 'addition'.  The airmasses should be 1.6, 1.7, 1.8, 1.9, or 2.0.  In the case of addition, you can set airmass_to_use to an array of airmasses and it will add all of them.  NOTE: Make sure, if you're running multiple experiments, to run the cell above for each one so you don't overwrite the wrong airmasses array.

In [ ]:
#GTR: Let's not do that experiment any more and just explore the different opSims.  
#So either take this out or just leave the array blank.

In [ ]:
experiment_to_run = 'addition'
#experiment_to_run = 'substitution'
#experiment_to_run = 'addition'
airmass_to_use = []

In [ ]:
if experiment_to_run == 'colors':
    save_file_name = 'AstroMetric_Colors_noDCR.npz'

In [ ]:
if experiment_to_run == 'substitution':    
    airmass_to_substitute = airmass_to_use[0]
    index_of_lowest = np.argmin(airmasses)
    airmasses[index_of_lowest] = airmass_to_substitute
    save_file_name = 'AstroMetric_SubstitutionDCR_' + str(int(airmass_to_substitute*10)) + '.npz'

In [ ]:
if experiment_to_run == 'addition':
    filters_to_add = np.tile('g', int(len(airmass_to_use)))
    airmasses = np.append(airmasses, airmass_to_use)
    filters = np.append(filters, filters_to_add)
    save_file_name = 'AstroMetric_TwilightDCR_' + str([int(airmass_to_use[i]*10) for i in range(len(airmass_to_use))]) + '.npz'

In [ ]:
#GTR: Not sure why this is here
#and not clear that this file name is being used
#I think that Bee was just trying to compare the results after 20 and 3 observations.

In [ ]:
#airmass removal cell
print(len(airmasses))
#if you don't want to remove any, set number_to_leave to "all"
number_to_leave = 20
number_to_leave="all"
if number_to_leave != "all":
    save_file_name = save_file_name[:-4] + "_" + str(number_to_leave) + "obs" + save_file_name[-4:]
    print("file name is " + save_file_name)
    number_to_remove = len(airmasses) - number_to_leave
else:
    number_to_remove = 0
removed = 0
while removed < number_to_remove:
    remove_index = random.randint(0,len(airmasses)-1)
    airmasses = np.delete(airmasses, remove_index)
    filters = np.delete(filters, remove_index)
    removed += 1 

138


In [ ]:
print(len(airmasses))
print(airmasses)
print(filters)
print(save_file_name)

138
[1.00622878 1.00853972 1.01967367 1.01444427 1.00743023 1.03187831
 1.02394573 1.00923144 1.0313592  1.00504214 1.18335984 1.02369731
 1.07047894 1.01751372 1.0176494  1.00222458 1.00719521 1.00393647
 1.00904816 1.03125394 1.07130849 1.01943226 1.01112162 1.01790893
 1.0139087  1.00696787 1.04467149 1.00689994 1.00877272 1.02842539
 1.01824907 1.01117482 1.02434356 1.013733   1.01019995 1.00620763
 1.04033332 1.00259324 1.08969973 1.02348594 1.00390752 1.04807564
 1.01139731 1.06130187 1.00223335 1.03758085 1.00631991 1.00661383
 1.01090133 1.01637917 1.01998853 1.02433815 1.0250975  1.0187453
 1.0445431  1.00717106 1.01155884 1.01358896 1.00267592 1.06361743
 1.01787706 1.01890238 1.02765961 1.02014511 1.05615775 1.13835903
 1.02252358 1.01385617 1.00496107 1.04433465 1.05973264 1.03202643
 1.02071543 1.0080826  1.0395498  1.31922158 1.07229784 1.01912032
 1.0591391  1.00499052 1.01146673 1.00523885 1.01122037 1.00265338
 1.0049515  1.04461183 1.0024018  1.00380577 1.00555118 1.0

In [ ]:
#GTR: I think that this is just to provide a basis of comparison with just a few (here 3) epochs.

In [ ]:
airmasses_20 = airmasses
filters_20 = filters
if experiment_to_run == 'addition':
    filters_to_add = np.tile('g', int(len(airmass_to_use)))
    airmasses = np.append(airmasses, airmass_to_use)
    filters = np.append(filters, filters_to_add)
    save_file_name = 'AstroMetric_TwilightDCR_' + str([int(airmass_to_use[i]*10) for i in range(len(airmass_to_use))]) + '.npz'
number_to_leave = 3
if number_to_leave != "all":
    save_file_name = save_file_name[:-4] + "_" + str(number_to_leave) + "obs" + save_file_name[-4:]
    print("file name is " + save_file_name)
    number_to_remove = len(airmasses) - number_to_leave
else:
    number_to_remove = 0
removed = 0
while removed < number_to_remove:
    remove_index = random.randint(0,len(airmasses)-1)
    airmasses = np.delete(airmasses, remove_index)
    filters = np.delete(filters, remove_index)
    removed += 1 
airmasses_3 = airmasses
filters_3 = filters


file name is AstroMetric_TwilightDCR_[]_3obs.npz


## generate observed slopes from true slopes and observations

In [ ]:
#BEE: lnlike calculates the loglikelihood, lnprior creates a prior on our linear fits, lnprob adds the prior to lnlike
#BEE: run_fit runs the mcmc walkers over a range of linear fits and selects the median as the best fit and half the 
#     difference between 16th and 84th percentiles as the error
#GTR: run_fit is computing the slope in the offset vs. tanZ plane for a single object

In [ ]:
def lnlike(theta, x, y, yerr):
    m, lnf = theta
    model = m*x
    inv_sigma2 = 1.0/(yerr**2. + model**2.*np.exp(2.*lnf))
    return -0.5*(np.sum(((y-model)**2.*inv_sigma2 - np.log(inv_sigma2))))

def lnprior(theta):
    m, lnf = theta
    if (-1.0 < m < 1.0) and (-100.0 < lnf < 100.0):
        return 0.0
    return -np.inf

def lnprob(theta, x, y, yerr):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, x, y, yerr)

def run_fit(tanZList, RList, RerrList):
    nll = lambda *args: -lnprob(*args)
    x = np.copy(tanZList)
    y = np.copy(RList)
    yerr = np.copy(RerrList)
    #first do a simple minimization to get starting values for mcmc
    pm = np.random.choice([-1.0,1.0], size=len(x), replace=True)
    result = minimize(nll, [-0.001, np.log(0.5)], args=(x, y, yerr))
    m_ml, lnf_ml = result["x"]
    #now run mcmc
    ndim, nwalkers = 2, 100
    pos = [result["x"] + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(x, y, yerr))
    sampler.run_mcmc(pos, 500)
    samples = sampler.chain[:, 50:, :].reshape((-1, ndim))
    ms = samples[np.random.randint(len(samples), size=100)][:,0]
    # return the median walker as the best slope and the half the 16-84th percentiles as the error
    m_mcmc, lnf_mcmc = map(lambda v: (v[1]), zip(*np.percentile(samples, [16, 50, 84], axis=0)))
    merr_mcmc, lnf_mcmc = map(lambda v: (0.5*(v[2]-v[0])), zip(*np.percentile(samples, [16, 50, 84], axis=0)))
    return m_mcmc, merr_mcmc

GTR: Split out cells that define functions from cells that make calls to those functions.

In [ ]:
#GTR: dcrSlopeCalc is computing the slope in the offset vs. tanZ plane for all the objects, calling run_fit for each

In [ ]:
def dcrSlopeCalc(airmasses, filters, test_quasars, makePlot = True):
    astrometric_error = [0.035, 0.025]
    obs_slopes_u    = np.zeros((len(test_quasars)))
    obs_slopes_uerr = np.zeros((len(test_quasars)))
    obs_slopes_g    = np.zeros((len(test_quasars)))
    obs_slopes_gerr = np.zeros((len(test_quasars)))
    imgNumString = 0
    xAxis = np.linspace(0, 2.0, 100)
    for i in range(len(test_quasars)):
        true_slope_u = test_quasars['u-slope'][i]
        true_slope_g = test_quasars['g-slope'][i]
    
        tanZList_u = np.array([])
        RerrList_u = np.array([])
        RList_u = np.array([])
        tanZList_g = np.array([])
        RerrList_g = np.array([])
        RList_g = np.array([])
    
        for j, airmass in enumerate(airmasses):
            tanZ_obs = np.tan(np.arccos(1.0/airmass)) #tangent of zenith angle of this observation
            if filters[j] == 'u':
                #calculate the observed offset
                #random scatter around the true offset using a normal distribution with the astrometric error as the standard deviation
                R_obs = normal(true_slope_u*tanZ_obs, astrometric_error[0])
                tanZList_u = np.append(tanZList_u, tanZ_obs)              #list of x axis values
                RerrList_u = np.append(RerrList_u, astrometric_error[0])  #list of y axis error values
                RList_u = np.append(RList_u, R_obs)                       #list of y axis values
            if filters[j] == 'g':
                R_obs = normal(true_slope_g*tanZ_obs, astrometric_error[1])
                tanZList_g = np.append(tanZList_g, tanZ_obs)
                RerrList_g = np.append(RerrList_g, astrometric_error[1])
                RList_g = np.append(RList_g, R_obs)
    
        # fit a stright line through the x and y values, using the y-err values
        m_mcmc_u, merr_mcmc_u = run_fit(tanZList_u, RList_u, RerrList_u)
        m_mcmc_g, merr_mcmc_g = run_fit(tanZList_g, RList_g, RerrList_g)
        if makePlot == True:
            bestFitLine_u = m_mcmc_u*xAxis + 0.0
            bestFitLine_g = m_mcmc_g*xAxis + 0.0
            trueFitLine_u = true_slope_u*xAxis + 0.0
            trueFitLine_g = true_slope_g*xAxis + 0.0
            plt.figure(figsize=(12,12))
            plt.subplot(121)
            plt.title('u-band observations + fit')
            plt.scatter(tanZList_u, RList_u, label = 'Observations')
            plt.plot(xAxis, bestFitLine_u, label='Fit Line')
            plt.plot(xAxis, trueFitLine_u, label = 'True Line')
            plt.legend()
            plt.xlabel('Tan(Z)')
            plt.ylabel('delta R')
            plt.xlim(0.0, 2.0)
            plt.scatter(x=tanZList_u, y=RList_u)
            plt.subplot(122)
            plt.title('g-band observations + fit')
            plt.scatter(tanZList_g, RList_g, label = 'Observations')
            plt.plot(xAxis, bestFitLine_g, label = 'Fit Line')
            plt.plot(xAxis, trueFitLine_g, label = 'True Line')
            plt.xlabel('Tan(Z)')
            plt.xlim(0.0, 2.0)
            plt.scatter(x=tanZList_g, y=RList_g)
            filename = "TanZimgFiles/airmassOffsetFit"+str(len(airmasses))+"_"+"{:0>5d}".format(imgNumString)
            plt.savefig(filename)
            plt.clf()
            plt.close()
            imgNumString += 1
        obs_slopes_u[i] = m_mcmc_u
        obs_slopes_uerr[i] = merr_mcmc_u
        obs_slopes_g[i] = m_mcmc_g
        obs_slopes_gerr[i] = merr_mcmc_g
    if makePlot == True:
        deltaSlope_u = []
        deltaSlope_g = []
        for i in range(len(obs_slopes_u)):
            deltaSlope_u = np.append(deltaSlope_u, test_quasars['u-slope'][i] - obs_slopes_u[i])
        for i in range(len(obs_slopes_g)):
            deltaSlope_g = np.append(deltaSlope_g, test_quasars['g-slope'][i] - obs_slopes_g[i])
        plt.figure(figsize=(12,12))
        plt.subplot(121)
        plt.hist(deltaSlope_u, bins=50, range=(-0.3,0.3))
        plt.title('Delta Slope u-band '+str(len(airmasses)))
        plt.subplot(122)
        plt.hist(deltaSlope_g, bins=50, range=(-0.3,0.3))
        plt.title('Delta Slope g-band '+str(len(airmasses)))
        filename = "DeltaSlopeimgFiles/deltaSlopeHist" + str(len(airmasses))
        plt.savefig(filename)
    return obs_slopes_u, obs_slopes_uerr, obs_slopes_g, obs_slopes_gerr

In [ ]:
#GTR: This cell actually calls the code that computes the slopes

In [ ]:
obs_slopes_u_20, obs_slopes_uerr, obs_slopes_g_20, obs_slopes_gerr = dcrSlopeCalc(airmasses_20, filters_20, test_quasars)
obs_slopes_u_3, obs_slopes_uerr, obs_slopes_g_3, obs_slopes_gerr = dcrSlopeCalc(airmasses_3, filters_3, test_quasars)


/anaconda3/lib/python3.7/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))


In [ ]:
sort_indices = np.argsort(test_quasars['zspec'])
plt.figure(figsize=(12,12))
plt.subplot(211)
plt.title('Observed DCR Slopes vs. Redshift')
plt.scatter(test_quasars['zspec'][sort_indices], test_quasars['u-slope'][sort_indices], color='red', label = 'True u slope')
plt.plot(test_quasars['zspec'][sort_indices], obs_slopes_u_20[sort_indices], color='black', label = 'Observed u slope@20 obs', alpha=0.7)
plt.plot(test_quasars['zspec'][sort_indices], obs_slopes_u_3[sort_indices], color='magenta',alpha=0.5, label = 'Observed u slope@3 obs')
plt.legend(loc='upper right')
plt.ylabel('u-band DCR slope')
plt.subplot(212)
plt.scatter(test_quasars['zspec'][sort_indices], test_quasars['g-slope'][sort_indices], color='blue', label = 'True g slope')
plt.plot(test_quasars['zspec'][sort_indices], obs_slopes_g_20[sort_indices], color='black', label = 'Observed g slope@20 obs', alpha=0.7)
plt.plot(test_quasars['zspec'][sort_indices], obs_slopes_g_3[sort_indices],color='cyan', alpha=0.5, label = 'Observed g slope@3 obs')
plt.legend(loc='upper right')
plt.ylabel('g-band DCR slope')
plt.xlabel('Redshift')


In [ ]:
#GTR: I have ignored everything past here.
#I was more concerned about making sure that we could reproduce the above plot.


## calculate redshift PDFs for observed quasars

In [28]:
def calculate_PDFs(parameters, zshifts, feature_zshift_fit, feature_covariance):
    
    num_features = int((np.shape(parameters)[0]-1)/2)
    num_of_quasars = np.shape(parameters)[1]
    
    #empty arrays to be filled
    feature_distance =  np.zeros((num_of_quasars, num_features, len(zshifts)))
    prob = np.zeros((num_of_quasars, len(zshifts)))
    chi_squared =  np.zeros((num_of_quasars, len(zshifts)))
    for i in range(num_of_quasars):
        #empty arrays to be filled
        features = np.zeros((num_features))
        covariance_matrix_of_features = np.zeros((num_features,num_features))
        
        # loop through all the features (e.g. 'u-g', 'g-r', 'r-i', 'i-z', 'u-slope', 'g-slope')
        for j in range(num_features):
            for k in range(num_features):
                if (j == k):
                    if j < 4:
                        # covaraince between the colors, on the diagonal
                        covariance_matrix_of_features[j,k] = parameters[j+num_features,i]**2.0 + parameters[j+num_features+1,i]**2.0
                    else:
                        # covaraince between the slopes, on the diagonal
                        covariance_matrix_of_features[j,k] = parameters[j+num_features+1,i]**2.0
                elif abs(j - k) == 1:
                    if j > k:
                        if j < 4:
                            # covaraince between the colors, just off the diagonal
                            covariance_matrix_of_features[j,k] = -1.0*parameters[j+num_features,i]**2.0
                    if k > j:
                        if k < 4:
                            # covaraince between the slopes, just off the diagonal
                            covariance_matrix_of_features[j,k] = -1.0*parameters[k+num_features,i]**2.0
            # difference between the features of this quasar and the regression calculate for all the quasars
            features[j] = parameters[j,i]
            feature_distance[i,j,:] = np.abs(features[j] - feature_zshift_fit[j,:])
        for z in range(len(zshifts)):
            # linear algebra from Weinstein et al. 2004
            A = np.matrix(feature_distance[i,:,z])
            B = np.matrix(covariance_matrix_of_features[:,:])
            C = np.matrix(feature_covariance[:,:,z])
            chi_squared[i,z] = np.dot(np.dot(A, (B + C).I), A.T)
            try:
                prob[i,z] = (np.exp(-1.0*chi_squared[i,z]/2.0))/(4.0*(math.pi**2.0)*(np.linalg.det(B + C)**0.5))
                #if np.isnan(prob[i,z]):
                    #prob[i,z] = 1e-250
                    #prob[i,z] = (np.finfo(np.float64).tiny)
            except:
                prob[i,z] = 0.0
        # normalize the probabilities
        sum_of_array = np.nansum(prob[i,:], axis=0, dtype=np.float64)
        try:
            prob[i,:] = prob[i,:]/sum_of_array
        except:
            prob[i,:] = 0.0*prob[i,:]
    return prob

In [29]:
#calculate the pdf of the redshift
if experiment_to_run != 'colors':
    obs_photoz_PDFs = calculate_PDFs(obs_parameters, zshifts, color_fit_dcr, color_covariance_dcr)
else:
    obs_photoz_PDFs = calculate_PDFs(obs_parameters, zshifts, color_fit, color_covariance)
'''
#dcr of opsim alone pdf
obs_photoz_PDFs_dcr1 = calculate_PDFs(obs_parameters_dcr1, zshifts, color_fit_dcr, color_covariance_dcr)
#dcr of opsim+longer observation time
obs_photoz_PDFs_dcr2 = calculate_PDFs(obs_parameters_dcr2, zshifts, color_fit_dcr, color_covariance_dcr)
#dcr of opsim+twilight survey
obs_photoz_PDFs_dcr3 = calculate_PDFs(obs_parameters_dcr3, zshifts, color_fit_dcr, color_covariance_dcr)
'''

NameError: name 'obs_parameters' is not defined

## calculate the peaks of the redshift PDFs

In [ ]:
def photozPDF_to_pointestimate(photoz_PDFs, zshifts):
    prob_threshold = 1.0/len(photoz_PDFs[0,:]) #threshold is above if all the probability were equally distributed
    num_of_quasars = len(photoz_PDFs[:,0])
    photoz_peaks = np.zeros((num_of_quasars))
    for i in range(num_of_quasars):
        zpeaks = np.array([])
        zprobs = np.array([])
        # all the non-nan values
        good_idxs = np.arange(len(photoz_PDFs[i,:]), dtype=np.int)[~np.isnan(photoz_PDFs[i,:])]
        # all the non-nan values above the probability threshold
        good_idxs_high = good_idxs[np.where(photoz_PDFs[i,:][~np.isnan(photoz_PDFs[i,:])] > prob_threshold)[0]]
        above_prob_threshold = list(good_idxs_high)
        # only find peaks if there is a value above the threshold
        if len(above_prob_threshold[1:-1]) > 1:
            # find all the contiguous bins above the probability threshold, these are the bumps in the PDF
            ranges = sum((list(t) for t in zip(above_prob_threshold, above_prob_threshold[1:]) if t[0]+1 != t[1]), [])
            # add the edges of the redshift range back on
            iranges = above_prob_threshold[0:1] + ranges + above_prob_threshold[-1:]
            # find the peak of each of the bumps
            for peaks in range(int(len(iranges)/2)):
                peak_zmin = iranges[int(peaks*2):int(peaks*2) + 2][0]
                peak_zmax = iranges[int(peaks*2):int(peaks*2) + 2][1]
                peak_maxprob = zshifts[peak_zmin:peak_zmax+1][np.argmax(photoz_PDFs[i,peak_zmin:peak_zmax+1])]
                # only count the peak if it isn't the minimum or maximum redshift bin
                # there can be weird edge effects in the PDFs, so we don't want those peaks
                if (peak_maxprob != zshifts[0]) and (peak_maxprob != zshifts[-1]):
                    zpeaks = np.append(zpeaks, peak_maxprob)
                    # the probability of that peak is all the area under the bump
                    zprobs = np.append(zprobs, np.sum(photoz_PDFs[i,peak_zmin:peak_zmax+1]))
                else:
                    zpeaks = np.append(zpeaks, peak_maxprob)
                    zprobs = np.append(zprobs, 0.0)
            photoz_peaks[i] = zpeaks[np.argmax(zprobs)]
        else:
            photoz_peaks[i] = np.nan
    return photoz_peaks

In [ ]:
obs_photoz_peaks = photozPDF_to_pointestimate(obs_photoz_PDFs, zshifts)
#obs_photoz_peaks_dcr1 = photozPDF_to_pointestimate(obs_photoz_PDFs_dcr1, zshifts)
#obs_photoz_peaks_dcr2 = photozPDF_to_pointestimate(obs_photoz_PDFs_dcr2, zshifts)
#obs_photoz_peaks_dcr3 = photozPDF_to_pointestimate(obs_photoz_PDFs_dcr3, zshifts)
print(obs_photoz_peaks)

## Save Experiment

In [ ]:
fileName = save_file_name
test_quasars_zspec = test_quasars['zspec']
if experiment_to_run != 'colors':
    np.savez(fileName, 
             airmasses=airmasses,
             filters=filters,
             deltaSlope_g=deltaSlope_g, 
             deltaSlope_u=deltaSlope_u, 
             z_phot=obs_photoz_peaks,
             z_true=test_quasars_zspec,
             redshift=zshifts)
else:
    np.savez(fileName,
            z_phot = obs_photoz_peaks,
            z_true = test_quasars_zspec,
            redshift=zshifts)

## write out the simulated quasars

In [ ]:
test_quasars_zspec = test_quasars['zspec']

with open('simulatedquasars_photozPDFs.dat', "w") as file_name:
    file_name.write("#zspec photozpeak photozPDF")
    file_name.write("\n")
    for i in range(len(test_quasars_zspec)):
        file_name.write("%0.4f %0.4f " % (test_quasars_zspec[i], obs_photoz_peaks[i]))
        for j in range(len(obs_photoz_PDFs[i,:])):
            file_name.write("%0.4f " % (obs_photoz_PDFs[i,j]))
        file_name.write("\n")

with open('simulatedquasars_obsparameters.dat', "w") as file_name:
    file_name.write("#zspec u-g g-r r-i i-z u-slope g-slope uerr gerr rerr ierr zerr u-slopeerr g-slopeerr")
    file_name.write("\n")
    for i in range(len(test_quasars_zspec)):
        for j in range(len(obs_parameters[:,i])):
            file_name.write("%0.4f " % (obs_parameters[j,i]))
        file_name.write("\n")

GTR: Have everything below read in data files in order to produce plots.  Let's just make single panels instead of 2x2.  We can build those if need be.

GTR: Add z_spec vs. zphot plots and Delta z histograms

## calculate the redshift quality metric

In [ ]:
def photo_z_robust_stdev(z_est, z_true, zshifts):
    """
    Sort the delta_z data into redshift bins in z_true.
    Delta_z is defined as (z_true - z_est) / (1. + z_true).
    
    Calculate the robust standard deviation in each bin as a function of true redshift.
    Robust standard deviation is defined as the standard deviation of delta_z in the bin where delta_z
    is defined as (z_true - z_est) / (1. + z_true) and we trim the highest and lowest 25% of delta_z values.
    """

    delta_z = (z_true - z_est) / (1. + z_true)
    idx_sort = z_true.argsort()
    delta_z_sort = delta_z[idx_sort]
    z_true_sort = z_true[idx_sort]
    idx_bins = z_true_sort.searchsorted(zshifts)
    delta_z_binned = [delta_z_sort[idx_bins[i]:idx_bins[i+1]] for i in range(len(zshifts)-1)]
    stdev_iqr_results = []
    for delta_z_data in delta_z_binned:
        if len(delta_z_data) == 0:
            stdev_iqr_results.append(np.nan)
            continue
        bin_25 = np.percentile(delta_z_data, 25.)
        bin_75 = np.percentile(delta_z_data, 75.)
        diff = bin_75 - bin_25
        stdev_iqr_results.append(diff/1.349)
    return np.array(stdev_iqr_results)

## Load in Save File

In [ ]:
#put the name of the file you want to plot from here
#file_to_load = 'this_is_a_placeholder.npz'  #Defaults to file that was just created, but can be changed
file_to_load = save_file_name
#file_to_load = "AstroMetric_TwilightDCR_[]_2obs.npz"
plot_data = np.load(file_to_load)
print(file_to_load[:-4])

In [ ]:
#calculate standard deviation of zphot over the interquartile range
stdev_iqr = photo_z_robust_stdev(plot_data['z_phot'], plot_data['z_true'], plot_data['redshift'])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=[10,10])
plt.xlabel('True Redshift')
plt.ylabel('Standarad Deviation within Interquartile Range')
plt.xlim(0.3,4)
plt.ylim(0,0.4)
plt.scatter(plot_data['redshift'][:-1], stdev_iqr)
plot_save_name = file_to_load[:-4] + '_stdev_iqr_plot.pdf'
plt.savefig(plot_save_name)

In [ ]:
plt.figure(figsize=[10,10])
plt.xlabel('True Redshift')
plt.ylabel('Zphot')
plt.scatter(plot_data['z_true'], plot_data['z_phot'])
plot_save_name = file_to_load[:-4] + '_ztrue_vs_zphot_plot.pdf'
plt.savefig(plot_save_name)

In [ ]:
deltaZ = np.subtract(plot_data['z_true'], plot_data['z_phot'])
data, bin_edges = np.histogram(deltaZ, bins='fd')
bins = 0.5*(bin_edges[:-1]+bin_edges[1:])
#z_err = np.divide(deltaZ, [1+z for z in plot_data['z_true']])
plt.figure(figsize=[10,10])
plt.xlabel('deltaZ')
plt.ylabel('Counts')
plt.step(bins,data)
plot_save_name = file_to_load[:-4] + '_deltaZ_hist_plot.pdf'
plt.savefig(plot_save_name)

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.hist(plot_data['deltaSlope_u'], bins=75, range=(-0.3,0.3))
plt.title('Delta Slope u-band '+str(len(plot_data['airmasses'])))
plt.subplot(122)
plt.hist(plot_data['deltaSlope_g'], bins=75, range=(-0.3,0.3))
plt.title('Delta Slope g-band '+str(len(plot_data['airmasses'])))
filename = "DeltaSlopeimgFiles/deltaSlopeHist" + str(len(plot_data['airmasses']))
plt.savefig(filename)